# **Content Based Collaborative Filtering**

In [1]:
import pandas as pd
import numpy as np

In [3]:
dataframe_robust_ratings = pd.read_csv('Preprocessed/robust_scaled_ratings.csv')
LDA_dataframe_jokes = pd.read_csv('Preprocessed/LDA_dataframe_jokes.csv')

## **Functions**

In [4]:
def get_intrested_jokes(user_id,df,jokes_df,no_jokes,interest_threshold,joke_matpic):
    """
    Finds the top "no_jokes" intrested jokes for active user

    Args:
        user_id (int): id of active user
        df (pd.Dataframe): Dataframe of joke_ratings for all users(normalized)
        jokes_df (pd.DataFrame): Dataframe of jokes after cleaning
        no_jokes (int): no of jokes
        interest_threshold (float b/w -1 to 1): rating threshold of intrest
        joke_matpic (pd.DataFrame): Dataframe of joke main topics

    Returns:
        list<int>: list of joke ids of the top "no_jokes" intrested jokes for active user
    """
    user_rating=df.loc[user_id-1]
    # generation of intrested topics of jokes for the user
    interested_topics=set([joke_matpic.loc[column-2] for column in df.columns[2:102] if(user_rating[column]<interest_threshold)])
    
    # finding new jokes for recommending
    new_jokes=[] # intrested and unrated
    old_jokes=[] # intrested and rated
    other_jokes=[] # unintrested and unrated
    worst_jokes=[] # unintrested and rated
    for i in range(100):
        if jokes_df['main_topic'][i] in interested_topics:
            if user_rating[i+2] !=user_rating[i+2]:
                new_jokes.append(i)
            else:
                old_jokes.append(i)
        elif user_rating[i+2] !=user_rating[i+2]:
            other_jokes.append(i)
        else:
            worst_jokes.append(i)
            
    #add new_jokes if needed
    if(len(new_jokes)<no_jokes):
        new_jokes+=old_jokes
        if(len(new_jokes)<no_jokes):
            if(no_jokes-len(new_jokes)>len(other_jokes)):
                new_jokes+=other_jokes
                new_jokes+=(sorted(worst_jokes,key=(lambda x: user_rating[x+2]))[::-1])[:no_jokes-len(new_jokes)]
            else:
                for i in np.random.choice(len(other_jokes),no_jokes-len(new_jokes),replace=False):
                    new_jokes.append(other_jokes[i])
                
    
    # choose randomly "no_jokes" no of jokes
    for i in np.random.choice(len(new_jokes), no_jokes, replace=False):
        new_jokes.append(new_jokes[i])
    return new_jokes[(-1*no_jokes):]
            
def recommend_cbf(user_id,df,jokes_df):
    """
    Recommends jokes based on cbf approach for given user
    
    Args:
        user_id (int): id of active user
        df (pd.Dataframe): Dataframe of joke_ratings for all users(normalized)
        jokes_df (pd.DataFrame): Dataframe of jokes after cleaning
    Returns:
        None
    """
    no_jokes=20 # no of jokes to recommend
    interest_threshold=0.8 # threshold for chossing rating of interest
    joke_matpic=jokes_df['main_topic'] # main topics of joke for finding interested topics of user 
    new_jokes=get_intrested_jokes(user_id,df,jokes_df,no_jokes,interest_threshold,joke_matpic)
    
    print("Recommending Jokes for user {}".format(user_id))
    for i,k in zip(range(no_jokes),new_jokes):
        print("Joke {}: {}".format(i+1,jokes_df['joke'][k]))

## **Results**

In [12]:
user_id = 68388
recommend_cbf(user_id,dataframe_robust_ratings,LDA_dataframe_jokes)

Recommending Jokes for user 68388
Joke 1: A couple has been married for 75 years. For the husband's 95thbirthday, his wife decides to surprise him by hiring a prostitute.That day, the doorbell rings. The husband uses his walker to get tothe door and opens it. A 21-year-old in a latex outfit smiles andsays, "Hi, I here to give you super sex!" The old man says, "I'll take the soup."
Joke 2: Q: How do you keep a computer programmer in the shower all day long?A: Give them a shampoo with a label that says"rinse, lather, repeat".
Joke 3: Two attorneys went into a diner and ordered two drinks.  Then they produced sandwiches from their briefcases and started to eat.  The owner becamequite concerned and marched over and told them, "You can't eat your ownsandwiches in here!"The attorneys looked at each other, shrugged their shoulders and thenexchanged sandwiches.
Joke 4: What do you call an American in the finals of the world cup?"Hey Beer Man!"
Joke 5: A woman has twins, and gives them up for a

## **Evaluation**